# Playing around with some stats

## Setup

In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [2]:
rename_dict = {
    'n_agents': 'Number of Agents',
    'p_rewiring': 'Probability of Rewiring',
    'uncertainty': 'Problem Easiness',
    'n_experiments': 'Number of Experiments',
    'share_of_correct_agents_at_convergence': 'Share of correct agents', # is this CORRECT ?!!?!?!?
    #'share_of_correct_agents_at_convergence':'Share of correct agents',
    'mean_degree': 'Mean Degree',
    'ba_degree':'BA-Degree',
    'convergence_step': 'Steps until convergence (log)',
    'approx_average_clustering_coefficient':'Clustering Coeff.',
    'average_degree': 'Average Degree',
    'degree_gini_coefficient': 'Degree Gini Coeff.',
    'avg_path_length': 'Avg. Path Length',
    'degree_entropy': 'Degree Entropy',
    'diameter': 'Diameter',
    'reachability_dominator_set_size': 'Reach. Dominator SS',
    'reachability_dominator_set_ratio': 'Reach. Dominator SR',
    'condensation_graph_size': 'Cond. Graph Size',
    'condensation_graph_ratio': 'Cond. Graph Ratio',
}

## New multiple linear regressions

In [ ]:
df_densify = pd.read_csv("data/simulation_densify.csv")
df_equalize = pd.read_csv("data/simulation_equalize.csv")

In [40]:
from itertools import chain, combinations
from sklearn.preprocessing import StandardScaler

def stats_subsets(
        df: pd.DataFrame, 
        fixed_predictors = ['uncertainty', 'n_experiments'], 
        main_predictors = ['degree_average', 'degree_gini', 'clustering_average'], 
        target = 'conclusion'
    ):

    # Generate all non-empty subsets of the first three predictors
    def all_subsets(lst):
        return chain.from_iterable(combinations(lst, r) for r in range(0, len(lst)+1))

    df_renamed = df.rename(columns={"degree_gini_coefficient": "gini", "approx_average_clustering_coefficient": "clustering"})
    
    results = []

    for subset in all_subsets(main_predictors):
        predictors = list(subset) + fixed_predictors
        X = df_renamed[predictors]
        y = df_renamed[target]

        # Normalize predictors and keep column names by converting back to a DataFrame
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=predictors, index=X.index)
        
        # Add intercept (preserves column names so statsmodels will show informative coef names)
        X_scaled_df = sm.add_constant(X_scaled_df, has_constant='add')
        
        # Fit OLS model
        model = sm.OLS(y, X_scaled_df).fit()
        
        # Build a tidy results table for this model
        params = model.params
        pvals = model.pvalues.reindex(params.index)
        results_df = pd.DataFrame({
            'Predictor': params.index,
            'Coefficient': params.values,
            'P-value': pvals.values
        })

        # Print formatted output: predictor table and R-squared
        print(f"Predictors: {predictors}")
        print(f"R-squared: {model.rsquared:.4f}")
        print(f"R-squared-adj: {model.rsquared_adj:.4f}\n")
        print(results_df.to_string(index=False, float_format='{:.6f}'.format))

        print('\n'+'-'*90+'\n')

        # Optionally store results for later
        # results.append({'predictors': predictors, 'results_df': results_df, 'r2': model.rsquared})


In [42]:
stats_subsets(df_equalize, main_predictors=['degree_gini', 'clustering_average'])

Predictors: ['uncertainty', 'n_experiments']
R-squared: 0.4978
R-squared-adj: 0.4968

    Predictor  Coefficient  P-value
        const     0.758795 0.000000
  uncertainty     0.050811 0.000000
n_experiments     0.028026 0.000000

------------------------------------------------------------------------------------------

Predictors: ['degree_gini', 'uncertainty', 'n_experiments']
R-squared: 0.5460
R-squared-adj: 0.5447

    Predictor  Coefficient  P-value
        const     0.758795 0.000000
  degree_gini    -0.018054 0.000000
  uncertainty     0.051708 0.000000
n_experiments     0.028247 0.000000

------------------------------------------------------------------------------------------

Predictors: ['clustering_average', 'uncertainty', 'n_experiments']
R-squared: 0.5075
R-squared-adj: 0.5060

         Predictor  Coefficient  P-value
             const     0.758795 0.000000
clustering_average    -0.008108 0.000010
       uncertainty     0.051104 0.000000
     n_experiments     0.028385

Notes:
- `equalize` keeps average degree fixed, but tries to maximally change Gini while keeping Clustering somewhat equal.
- Adding Gini to the bandit predictors increases the $R^2$ by about 5%.
- Adding Clustering to the bandit predictors increases the $R^2$ by about 1%.
- The coefficients of Gini and Clustering do not change much across multiple conditions. 
- The $R^2$ when including Gini is basically the same as the $R^2$ when including both Gini and Clustering.

In [43]:
stats_subsets(df_densify)

Predictors: ['uncertainty', 'n_experiments']
R-squared: 0.4082
R-squared-adj: 0.4070

    Predictor  Coefficient  P-value
        const     0.751346 0.000000
  uncertainty     0.043624 0.000000
n_experiments     0.020422 0.000000

------------------------------------------------------------------------------------------

Predictors: ['degree_average', 'uncertainty', 'n_experiments']
R-squared: 0.4502
R-squared-adj: 0.4485

     Predictor  Coefficient  P-value
         const     0.751346 0.000000
degree_average     0.015350 0.000000
   uncertainty     0.044400 0.000000
 n_experiments     0.021234 0.000000

------------------------------------------------------------------------------------------

Predictors: ['degree_gini', 'uncertainty', 'n_experiments']
R-squared: 0.4138
R-squared-adj: 0.4120

    Predictor  Coefficient  P-value
        const     0.751346 0.000000
  degree_gini    -0.005584 0.002130
  uncertainty     0.043568 0.000000
n_experiments     0.020351 0.000000

-------------

Notes:
- `densify` increases average degree, while trying to keep Gini and Clustering somewhat equal.
- Adding average-degree to the bandit predictors only increases the $R^2$ by roughly 4%.
- The coefficients of average-degree, Gini and Clustering do not change much across multiple conditions. 
- The $R^2$ when including average-degree is basically the same as the $R^2$ when including Gini and Clustering too.

## Measuring colinearity: it's pretty bad :(

In [3]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [5]:
df_densify = pd.read_csv("data/simulation_densify.csv")
df_equalize = pd.read_csv("data/simulation_equalize.csv")

In [9]:
def vif(
        df: pd.DataFrame, 
        columns_corr = ["degree_average", "degree_gini",'clustering_average']
    ):
    df_predictors = df[columns_corr]

    vif_data = pd.DataFrame()
    vif_data["feature"] = df_predictors.columns

    vif_data["VIF"] = [variance_inflation_factor(df_predictors.values, i)
                            for i in range(len(df_predictors.columns))]
    print(f"VIF Interpretation: \n"
          f"\tValues near 1 mean predictors are independent.\n"
          f"\tValues between 1 and 5 shows moderate correlation which is sometime acceptable.\n"
          f"\tValues greater than 5 represent critical levels of multicollinearity where the coefficients are poorly estimated, and the p-values are questionable.")
    print(vif_data)

In [7]:
columns_corr = ["degree_average", "degree_gini",'clustering_average']
df_densify[columns_corr].corr()

,degree_average,degree_gini,clustering_average
degree_average,1.000000,-0.158416,-0.044915
degree_gini,-0.158416,1.000000,-0.016626
clustering_average,-0.044915,-0.016626,1.000000


In [10]:
vif(df_densify)


VIF Interpretation: 
	Values near 1 mean predictors are independent.
	Values between 1 and 5 shows moderate correlation which is sometime acceptable.
	Values greater than 5 represent critical levels of multicollinearity where the coefficients are poorly estimated, and the p-values are questionable.
              feature          VIF
0      degree_average   145.918957
1         degree_gini  8031.835809
2  clustering_average  8137.962406


In [11]:
df_equalize[columns_corr].corr()

,degree_average,degree_gini,clustering_average
degree_average,NaN,NaN,NaN
degree_gini,NaN,1.0000,0.1708
clustering_average,NaN,0.1708,1.0000


In [12]:
vif(df_equalize, columns_corr=["degree_gini", "clustering_average"])

VIF Interpretation: 
	Values near 1 mean predictors are independent.
	Values between 1 and 5 shows moderate correlation which is sometime acceptable.
	Values greater than 5 represent critical levels of multicollinearity where the coefficients are poorly estimated, and the p-values are questionable.
              feature         VIF
0         degree_gini  616.798911
1  clustering_average  616.798911


## Other approaches: Ridge & Lasso

Lasso and Ridge regression are both regularization techniques used to prevent overfitting in linear regression models by adding a penalty term to the loss function.
 The key difference lies in the type of penalty applied: Ridge regression uses L2 regularization, which adds a penalty proportional to the sum of the squared coefficients, while Lasso regression uses L1 regularization, which adds a penalty proportional to the sum of the absolute values of the coefficients.

This fundamental difference leads to distinct behaviors. Ridge regression shrinks the coefficients of less important features toward zero but does not set them exactly to zero, meaning all predictors remain in the model.
 This approach is beneficial when all features are potentially relevant and the goal is to reduce overfitting without eliminating variables.
 In contrast, Lasso regression can set some coefficients exactly to zero, effectively performing automatic feature selection by excluding irrelevant or redundant predictors from the model.
 This results in a simpler, more interpretable model with fewer features.

The geometric interpretation explains this behavior: the L1 constraint in Lasso creates a diamond-shaped boundary with corners, making it more likely for the solution to land on a corner where a coefficient is zero. The L2 constraint in Ridge forms a circular boundary, which is rotationally invariant and less likely to produce zero coefficients.
 Consequently, Lasso is preferred when feature selection is important or when a sparse solution is desired, such as in high-dimensional data where only a few features are expected to be relevant. Ridge is more suitable when all features are believed to contribute to the outcome, such as in predicting house prices where multiple factors like size, location, and number of bedrooms are all potentially important.

Lasso can be more sensitive to outliers due to the absolute value in its penalty term, while Ridge is generally more robust.
 Computationally, Ridge regression is typically faster as it does not involve feature selection, whereas Lasso may be slower due to the need to identify which coefficients to set to zero.
 Both methods require tuning a hyperparameter (lambda) to control the strength of regularization.
 In cases where features are correlated, Elastic Net, which combines both L1 and L2 penalties, can be advantageous by balancing feature selection and handling of correlated predictors.

Conclusion
- For densify: LASSO selects Average Degree as the most relevant network feature. It consistently drives Gini and Clustering to zero, as expected. 
- For equalize: LASSO selects Gini as the most relevant network feature. It consistently drives Clustering to zero, as expected. 

In [24]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from itertools import chain, combinations

### Ridge

In [ ]:
def ridge_subsets(
    df: pd.DataFrame, 
    fixed_predictors = ['uncertainty', 'n_experiments'], 
    main_predictors = ['degree_average', 'degree_gini', 'clustering_average'], 
    target = 'conclusion',
    alpha=1.0 # Regularization strength
):
    """
    Fits Ridge regression models for all subsets of main_predictors.
    """

    # Generate all non-empty subsets
    def all_subsets(lst):
        # Includes the empty subset for completeness, which results in fixed_predictors only
        return chain.from_iterable(combinations(lst, r) for r in range(0, len(lst)+1))
    
    for subset in all_subsets(main_predictors):
        predictors = list(subset) + fixed_predictors
        X = df[predictors]
        y = df[target]

        # Normalize predictors
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=predictors, index=X.index)
        
        # Fit Ridge model
        # NOTE: Ridge and Lasso automatically include an intercept (constant) by default.
        model = Ridge(alpha=alpha, random_state=42) # random_state for reproducibility
        model.fit(X_scaled_df, y)
        
        # Extract results
        # Intercept is stored separately in sklearn
        coef_names = ['Intercept'] + predictors
        coefficients = [model.intercept_] + list(model.coef_)
        r_squared = model.score(X_scaled_df, y) # R^2 on the training data
        
        # Build a tidy results table for this model
        results_df = pd.DataFrame({
            'Predictor': coef_names,
            'Coefficient': coefficients,
        })

        # Print formatted output: predictor table and R-squared
        print(f"--- Ridge Regression (alpha={alpha}) ---")
        print(f"Predictors: {predictors}")
        print(f"R-squared (Train): {r_squared:.4f}")
        print(f"Number of non-zero coefficients: {sum(1 for c in model.coef_ if abs(c) > 1e-9)}") # Always equal to len(predictors) in Ridge
        print('\n' + results_df.to_string(index=False, float_format='{:.6f}'.format))

        print('\n'+'-'*90+'\n')

In [28]:
ridge_subsets(df_densify, alpha=1.0)

--- Ridge Regression (alpha=1.0) ---
Predictors: ['uncertainty', 'n_experiments']
R-squared (Train): 0.4082
Number of non-zero coefficients: 2

    Predictor  Coefficient
    Intercept     0.751346
  uncertainty     0.043580
n_experiments     0.020401

------------------------------------------------------------------------------------------

--- Ridge Regression (alpha=1.0) ---
Predictors: ['degree_average', 'uncertainty', 'n_experiments']
R-squared (Train): 0.4502
Number of non-zero coefficients: 3

     Predictor  Coefficient
     Intercept     0.751346
degree_average     0.015331
   uncertainty     0.044354
 n_experiments     0.021210

------------------------------------------------------------------------------------------

--- Ridge Regression (alpha=1.0) ---
Predictors: ['degree_gini', 'uncertainty', 'n_experiments']
R-squared (Train): 0.4138
Number of non-zero coefficients: 3

    Predictor  Coefficient
    Intercept     0.751346
  degree_gini    -0.005579
  uncertainty     0.

In [29]:
ridge_subsets(df_equalize, main_predictors=['degree_gini', 'clustering_average'], alpha=1.0)

--- Ridge Regression (alpha=1.0) ---
Predictors: ['uncertainty', 'n_experiments']
R-squared (Train): 0.4978
Number of non-zero coefficients: 2

    Predictor  Coefficient
    Intercept     0.758795
  uncertainty     0.050760
n_experiments     0.027998

------------------------------------------------------------------------------------------

--- Ridge Regression (alpha=1.0) ---
Predictors: ['degree_gini', 'uncertainty', 'n_experiments']
R-squared (Train): 0.5460
Number of non-zero coefficients: 3

    Predictor  Coefficient
    Intercept     0.758795
  degree_gini    -0.018033
  uncertainty     0.051655
n_experiments     0.028218

------------------------------------------------------------------------------------------

--- Ridge Regression (alpha=1.0) ---
Predictors: ['clustering_average', 'uncertainty', 'n_experiments']
R-squared (Train): 0.5075
Number of non-zero coefficients: 3

         Predictor  Coefficient
         Intercept     0.758795
clustering_average    -0.008097
      

### LASSO

This function uses L1 regularization via sklearn.linear_model.Lasso. LASSO is particularly useful for feature selection as it can drive the coefficients of less important predictors exactly to zero.

In [47]:
def lasso_subsets(
    df: pd.DataFrame, 
    fixed_predictors = ['uncertainty', 'n_experiments'], 
    main_predictors = ['degree_average', 'degree_gini', 'clustering_average'], 
    target = 'conclusion',
    alpha=0.01 # A smaller alpha is often a better starting point for Lasso
):
    """
    Fits LASSO regression models for all subsets of main_predictors.
    """
    # Generate all non-empty subsets
    def all_subsets(lst):
        # Includes the empty subset for completeness, which results in fixed_predictors only
        return chain.from_iterable(combinations(lst, r) for r in range(0, len(lst)+1))

    for subset in all_subsets(main_predictors):
        predictors = list(subset) + fixed_predictors
        X = df[predictors]
        y = df[target]

        # Normalize predictors
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        X_scaled_df = pd.DataFrame(X_scaled, columns=predictors, index=X.index)
        
        # Fit LASSO model
        # NOTE: We use a small alpha (e.g., 0.01) to allow for some coefficients to remain non-zero.
        model = Lasso(alpha=alpha, random_state=42, max_iter=10000) # Increased max_iter for robustness
        model.fit(X_scaled_df, y)
        
        # Extract results
        # Intercept is stored separately in sklearn
        coef_names = ['Intercept'] + predictors
        coefficients = [model.intercept_] + list(model.coef_)
        r_squared = model.score(X_scaled_df, y) # R^2 on the training data
        
        # Build a tidy results table for this model
        results_df = pd.DataFrame({
            'Predictor': coef_names,
            'Coefficient': coefficients,
        })

        # Print formatted output: predictor table and R-squared
        print(f"--- LASSO Regression (alpha={alpha}) ---")
        print(f"Predictors: {predictors}")
        print(f"R-squared (Train): {r_squared:.4f}")
        # Count non-zero coefficients. LASSO is known for setting coefficients exactly to zero.
        print(f"Number of non-zero coefficients: {sum(1 for c in model.coef_ if abs(c) > 1e-9)}") 
        print('\n' + results_df.to_string(index=False, float_format='{:.6f}'.format))

        print('\n'+'-'*90+'\n')

In [32]:
lasso_subsets(df_densify, alpha=0.01)

--- LASSO Regression (alpha=0.01) ---
Predictors: ['uncertainty', 'n_experiments']
R-squared (Train): 0.3716
Number of non-zero coefficients: 2

    Predictor  Coefficient
    Intercept     0.751346
  uncertainty     0.033392
n_experiments     0.010191

------------------------------------------------------------------------------------------

--- LASSO Regression (alpha=0.01) ---
Predictors: ['degree_average', 'uncertainty', 'n_experiments']
R-squared (Train): 0.3916
Number of non-zero coefficients: 3

     Predictor  Coefficient
     Intercept     0.751346
degree_average     0.004257
   uncertainty     0.033608
 n_experiments     0.010416

------------------------------------------------------------------------------------------

--- LASSO Regression (alpha=0.01) ---
Predictors: ['degree_gini', 'uncertainty', 'n_experiments']
R-squared (Train): 0.3716
Number of non-zero coefficients: 2

    Predictor  Coefficient
    Intercept     0.751346
  degree_gini    -0.000000
  uncertainty    

Notes:
- LASSO consistently drives Gini and Clustering to zero, as expected.

In [33]:
lasso_subsets(df_equalize, main_predictors=['degree_gini', 'clustering_average'], alpha=0.01)

--- LASSO Regression (alpha=0.01) ---
Predictors: ['uncertainty', 'n_experiments']
R-squared (Train): 0.4680
Number of non-zero coefficients: 2

    Predictor  Coefficient
    Intercept     0.758795
  uncertainty     0.040771
n_experiments     0.017986

------------------------------------------------------------------------------------------

--- LASSO Regression (alpha=0.01) ---
Predictors: ['degree_gini', 'uncertainty', 'n_experiments']
R-squared (Train): 0.4995
Number of non-zero coefficients: 3

    Predictor  Coefficient
    Intercept     0.758795
  degree_gini    -0.007407
  uncertainty     0.041139
n_experiments     0.018077

------------------------------------------------------------------------------------------

--- LASSO Regression (alpha=0.01) ---
Predictors: ['clustering_average', 'uncertainty', 'n_experiments']
R-squared (Train): 0.4680
Number of non-zero coefficients: 2

         Predictor  Coefficient
         Intercept     0.758795
clustering_average    -0.000000
   

Notes:
- LASSO consistently drives Clustering to zero, as expected.

## Elastic Net

Note:
Elastic Net introduces bias by shrinking coefficient estimates toward zero through regularization (controlled by λ). This bias-variance trade-off reduces model variance and overfitting, often improving prediction accuracy. However, the magnitude of coefficients is underestimated, making them poor estimates of true effect sizes. While useful for feature selection and ranking, the coefficients should not be interpreted as unbiased measures of importance or causal impact.

In [52]:
from itertools import combinations, chain
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNetCV
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm

def stats_enet(
        df: pd.DataFrame, 
        fixed_predictors: list[str]= ['uncertainty', 'n_experiments'], 
        main_predictors: list[str] = ['degree_average', 'degree_gini', 'clustering_average'], 
        target: str = 'conclusion',
        n_bootstraps: int = 100,
        l1_ratio: float | list[float]= [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
        selection_threshold: float = 1e-6
    ) -> None:
    """
    Perform bootstrapped Elastic Net regression with variable selection over all subsets of main predictors.

    Fits ElasticNetCV models on bootstrap samples for every combination of main_predictors 
    (e.g., 'degree_average', 'degree_gini', 'clustering_average') combined with fixed_predictors 
    (e.g., 'uncertainty', 'n_experiments'). For each predictor set:
    
    - Scales features using StandardScaler
    - Uses 5-fold cross-validation to select optimal alpha and l1_ratio
    - Aggregates coefficient means and selection frequencies across bootstraps
    - Computes average R²

    Note: l1_ratio can be set to a value or an interval.
    
    Parameters
    ----------
    df : pd.DataFrame
        Input data containing predictors and target variable.
    fixed_predictors : list of str, default ['uncertainty', 'n_experiments']
        Predictors included in all models.
    main_predictors : list of str, default ['degree_average', 'degree_gini', 'clustering_average']
        Predictors to evaluate in all possible subsets.
    target : str, default 'conclusion'
        Name of the target variable column in df.
    n_bootstraps : int, default 100
        Number of bootstrap iterations for stability assessment.
    l1_ratio: float or list of float, default [0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1]
        L1 ratio(s) used in cross-validation step.
    selection_threshold : float, default 1e-6
        Absolute coefficient threshold to count a variable as selected.

    Outputs
    -------
    Prints for each predictor subset:
        - List of predictors
        - Average R² across bootstraps
        - Average R² (adjusted) placeholder (currently not computed)
        - Table of predictors, their average (normalized) coefficients, and selection frequency
    """   

    def all_subsets(lst):
        return chain.from_iterable(combinations(lst, r) for r in range(0, len(lst)+1))
    
    for subset in all_subsets(main_predictors):
        predictors = list(subset) + fixed_predictors
        X = df[predictors]
        y = df[target]

        # Store coefficient and selection count per predictor
        coef_sum = {pred: 0.0 for pred in predictors}
        selection_count = {pred: 0 for pred in predictors}
        r2_sum = 0.0
        r2_adj_sum = 0.0

        for _ in tqdm(range(n_bootstraps)):
            sample_idx = np.random.choice(X.index, size=len(X), replace=True)
            X_boot, y_boot = X.loc[sample_idx], y.loc[sample_idx]

            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X_boot)
            
            model = ElasticNetCV(
                cv=5, 
                random_state=42, 
                l1_ratio=l1_ratio, 
                max_iter=1000,
            )
            
            model.fit(X_scaled, y_boot)
            
            r2_sum += model.score(X_scaled, y_boot)
            # r2_adj_sum = model.(X_scaled, y_boot)

            for i, pred in enumerate(predictors):
                if abs(model.coef_[i]) > selection_threshold:
                    coef_sum[pred] += model.coef_[i]
                    selection_count[pred] += 1
                    

        # Compute average coefficient and frequency
        results = []
        for pred in predictors:
            avg_coef = coef_sum[pred] / n_bootstraps
            freq = selection_count[pred] / n_bootstraps
            results.append({'Predictor': pred, 'Coefficient': avg_coef, 'Frequency': freq})
        
        results_df = pd.DataFrame(results)
        print(f"Predictors: {predictors}")
        print(f"R-squared: {r2_sum/n_bootstraps:.4f}\n")
        print(f"R-squared (adjusted): {r2_adj_sum/n_bootstraps:.4f}\n")
        print(results_df.to_string(index=False, float_format='{:.6f}'.format))
        print('\n' + '-'*90 + '\n')   

In [50]:
stats_enet(df_densify)

  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['uncertainty', 'n_experiments']
R-squared: 0.4089

R-squared (adjusted): 0.0000

    Predictor  Coefficient  Frequency
  uncertainty     0.043271   1.000000
n_experiments     0.020261   1.000000

------------------------------------------------------------------------------------------



  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['degree_average', 'uncertainty', 'n_experiments']
R-squared: 0.4528

R-squared (adjusted): 0.0000

     Predictor  Coefficient  Frequency
degree_average     0.015338   1.000000
   uncertainty     0.044159   1.000000
 n_experiments     0.021601   1.000000

------------------------------------------------------------------------------------------



  0%|          | 0/100 [00:00<?, ?it/s]

...

Predictors: ['degree_average', 'clustering_average', 'uncertainty', 'n_experiments']
R-squared: 0.4514

R-squared (adjusted): 0.0000

         Predictor  Coefficient  Frequency
    degree_average     0.015033   1.000000
clustering_average    -0.001274   0.810000
       uncertainty     0.044350   1.000000
     n_experiments     0.020924   1.000000

------------------------------------------------------------------------------------------



  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['degree_gini', 'clustering_average', 'uncertainty', 'n_experiments']
R-squared: 0.4156

R-squared (adjusted): 0.0000

         Predictor  Coefficient  Frequency
       degree_gini    -0.005403   1.000000
clustering_average    -0.001644   0.830000
       uncertainty     0.043164   1.000000
     n_experiments     0.020115   1.000000

------------------------------------------------------------------------------------------



  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['degree_average', 'degree_gini', 'clustering_average', 'uncertainty', 'n_experiments']
R-squared: 0.4564

R-squared (adjusted): 0.0000

         Predictor  Coefficient  Frequency
    degree_average     0.014652   1.000000
       degree_gini    -0.003117   0.910000
clustering_average    -0.001228   0.820000
       uncertainty     0.044031   1.000000
     n_experiments     0.021005   1.000000

------------------------------------------------------------------------------------------



In [51]:
stats_enet(df_equalize, main_predictors=['degree_gini', 'clustering_average'])

  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['uncertainty', 'n_experiments']
R-squared: 0.4979

R-squared (adjusted): 0.0000

    Predictor  Coefficient  Frequency
  uncertainty     0.050303   1.000000
n_experiments     0.027791   1.000000

------------------------------------------------------------------------------------------



  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['degree_gini', 'uncertainty', 'n_experiments']
R-squared: 0.5467

R-squared (adjusted): 0.0000

    Predictor  Coefficient  Frequency
  degree_gini    -0.018080   1.000000
  uncertainty     0.051505   1.000000
n_experiments     0.028191   1.000000

------------------------------------------------------------------------------------------



  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['clustering_average', 'uncertainty', 'n_experiments']
R-squared: 0.5073

R-squared (adjusted): 0.0000

         Predictor  Coefficient  Frequency
clustering_average    -0.007792   1.000000
       uncertainty     0.051000   1.000000
     n_experiments     0.028172   1.000000

------------------------------------------------------------------------------------------



  0%|          | 0/100 [00:00<?, ?it/s]

Predictors: ['degree_gini', 'clustering_average', 'uncertainty', 'n_experiments']
R-squared: 0.5522

R-squared (adjusted): 0.0000

         Predictor  Coefficient  Frequency
       degree_gini    -0.016956   1.000000
clustering_average    -0.005444   0.990000
       uncertainty     0.051935   1.000000
     n_experiments     0.028359   1.000000

------------------------------------------------------------------------------------------

